In [1]:
import pandas as pd

In [2]:
data01 = pd.read_csv('/home/suneel/Desktop/chess-engine/Data/Data Lake/Chess_1.csv')
data02 = pd.read_csv('/home/suneel/Desktop/chess-engine/Data/Data Lake/Chess_2.csv')
data03 = pd.concat([data01,data02],axis=0)
data03

,FEN,Evaluation
0,r2q1rk1/pb3pbp/1ppp1np1/2nPp3/4P3/2P1B2P/PPBNN...,16
1,r7/ppQ1n2p/2n1k1p1/1N1p1q2/1P1P4/2P3P1/P7/2K5 ...,-929
2,r1bq1rk1/ppp1bppp/2n2n2/3Pp3/2B5/2PP1N1P/PP3PP...,4
3,6k1/R2b2r1/8/7R/2PPP3/8/7P/5K2 b - - 0 42,546
4,r4r2/pp4pp/k1nQ1q2/5P2/2B5/8/P5PP/1R3RK1 b - -...,684
...,...,...
1980494,rnb1k2r/pp3ppp/4pn2/3pB3/8/P2PqN2/2PKPbPP/RN1Q...,-402
1980495,6R1/pq1k1p2/1p1pp3/8/8/1P5r/P1PPQP1P/R5K1 w - ...,641
1980496,1r3rk1/pp4p1/1qn4p/4pp2/8/2NPPNP1/PP1Q1KbP/R7 ...,-357
1980497,rnb1k1r1/pp3pBp/2pqp1p1/3P4/8/P1N2N2/1PP2PPP/R...,686


In [3]:
data03 = data03.reset_index()

In [45]:
data04 = data03.copy()
mean = data04['Evaluation'].mean()
std = data04['Evaluation'].std()
data04['Evaluation_std'] = (data04['Evaluation'] - mean) / std
print("mean : ",mean)
print("std : ",std)
data04

mean :  13.409747727645131
std :  588.6268786929631


,index,FEN,Evaluation,Evaluation_std
0,0,r2q1rk1/pb3pbp/1ppp1np1/2nPp3/4P3/2P1B2P/PPBNN...,16,0.004400
1,1,r7/ppQ1n2p/2n1k1p1/1N1p1q2/1P1P4/2P3P1/P7/2K5 ...,-929,-1.601031
2,2,r1bq1rk1/ppp1bppp/2n2n2/3Pp3/2B5/2PP1N1P/PP3PP...,4,-0.015986
3,3,6k1/R2b2r1/8/7R/2PPP3/8/7P/5K2 b - - 0 42,546,0.904801
4,4,r4r2/pp4pp/k1nQ1q2/5P2/2B5/8/P5PP/1R3RK1 b - -...,684,1.139245
...,...,...,...,...
3960754,1980494,rnb1k2r/pp3ppp/4pn2/3pB3/8/P2PqN2/2PKPbPP/RN1Q...,-402,-0.705727
3960755,1980495,6R1/pq1k1p2/1p1pp3/8/8/1P5r/P1PPQP1P/R5K1 w - ...,641,1.066194
3960756,1980496,1r3rk1/pp4p1/1qn4p/4pp2/8/2NPPNP1/PP1Q1KbP/R7 ...,-357,-0.629278
3960757,1980497,rnb1k1r1/pp3pBp/2pqp1p1/3P4/8/P1N2N2/1PP2PPP/R...,686,1.142643


In [64]:
import numpy as np
def encoding(fen) :
    '''
    a (64 + (8)) * 1 vector
    ''' 
    power = {
        'K' : 100,'k' : -100,'P' : 1,'p' : -1,'B' : 3,'b' : -3,'N' : 3,'n' : -3,'R' :5,'r' : '-5','Q' : 9,'q' : -9 
    }
    position,active,castling,enpassant,halfmove,fullmove = fen.split(' ')
    encoded = np.zeros((72,1))
    ranks = position.split('/')
    ind = 0
    for rank in ranks :
        j = 0
        while j < len(rank) :
            if rank[j] < 'A' :
                ind += int(rank[j])
            else :
                encoded[ind] = power[rank[j]]
                ind += 1
            j+=1
    encoded[ind] = 1 if active == 'w' else -1
    ind += 1
    for i in castling :
        if i != '-' :
            encoded[ind] = 1
        ind += 1
    #print(ind)
    encoded[ind] = int(halfmove)
    encoded[ind+1] = int(fullmove)
    encoded[ind+2] = 0 if enpassant == '-' else ((ord(enpassant[0]) - ord('a'))*8 + int(enpassant[1]) - 1) 
    return encoded


In [65]:
fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
#print(fen.split(' '))
print(encoding(fen))

[[  -5.]
 [  -3.]
 [  -3.]
 [  -9.]
 [-100.]
 [  -3.]
 [  -3.]
 [  -5.]
 [  -1.]
 [  -1.]
 [  -1.]
 [  -1.]
 [  -1.]
 [  -1.]
 [  -1.]
 [  -1.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   5.]
 [   3.]
 [   3.]
 [   9.]
 [ 100.]
 [   3.]
 [   3.]
 [   5.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   1.]
 [   0.]
 [   1.]
 [   0.]]


In [77]:
import torch
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(72,256),
    nn.BatchNorm1d(256),
    nn.ReLU(),

    nn.Linear(256,256),
    nn.BatchNorm1d(256),
    nn.ReLU(),

    nn.Linear(256,256),
    nn.BatchNorm1d(256),
    nn.ReLU(),

    nn.Linear(256,256),
    nn.BatchNorm1d(256),
    nn.ReLU(),

    nn.Linear(256,1)
)

In [78]:
print(model)

Sequential(
  (0): Linear(in_features=72, out_features=256, bias=True)
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Linear(in_features=256, out_features=256, bias=True)
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Linear(in_features=256, out_features=1, bias=True)
)


In [79]:
t1 = torch.tensor(encoding(fen)).squeeze()
print(t1.shape)

torch.Size([72])


In [85]:
from torch.utils.data import Dataset,DataLoader
import pandas as pd

class Chessloader(Dataset) :
    def __init__(self,df) :
        self.df = df
    def __len__(self) :
        return len(self.df)
    def __getitem__(self,ind) :
        row = self.df.iloc[ind]
        tensor = torch.tensor(encoding(row['FEN']),dtype=torch.float32).squeeze()
        target = torch.tensor(row['Evaluation_std'], dtype=torch.float32)
        return tensor,target


In [86]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(data04.sample(2000000,random_state=42),test_size=0.25,random_state=42)

In [87]:
train_set = Chessloader(train_data)
test_set = Chessloader(test_data)
train_loader = DataLoader(train_set,batch_size=32,shuffle=False)
test_loader  = DataLoader(test_set,batch_size=32,shuffle=False)

In [94]:
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm  # Import tqdm for the progress bar
import os

def train(train_loader, test_loader, model, num_epochs=20, lr=0.001, save_path="/home/suneel/Desktop/chess-engine/models/basemodel1/"):
    # Define the loss function and the optimizer
    loss_fn = nn.MSELoss()  # Use MSE loss for regression tasks
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    # Device configuration (move the model to GPU if available)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Create save directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Lists to store the losses
    train_losses = []
    test_losses = []

    # Variable to track the best test loss
    best_test_loss = float('inf')

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_train_loss = 0.0
        
        # Use tqdm to display a progress bar for the training loop
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            targets = targets.unsqueeze(-1)  # Ensure the target is the correct shape

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_fn(outputs, targets)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        # Calculate average training loss for this epoch
        avg_train_loss = running_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

        # Evaluate on the test set
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            running_test_loss = 0.0
            all_predictions = []
            all_actuals = []
            for inputs, targets in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Testing", leave=False):
                inputs, targets = inputs.to(device), targets.to(device)

                # Forward pass
                outputs = model(inputs)

                # Store predictions and actuals for later use
                all_predictions.append(outputs.cpu().numpy())
                all_actuals.append(targets.cpu().numpy())

                # Compute loss
                loss = loss_fn(outputs, targets)
                running_test_loss += loss.item()

            avg_test_loss = running_test_loss / len(test_loader)
            test_losses.append(avg_test_loss)
            print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {avg_test_loss:.4f}")

            # Flatten the lists of predictions and actuals
            all_predictions = np.concatenate(all_predictions, axis=0)
            all_actuals = np.concatenate(all_actuals, axis=0)

            # Print the last 5 predictions and actuals from the test set
            print("\nLast 5 Predictions and Actuals:")
            for i in range(5):
                print(f"Predicted: {all_predictions[-(i+1)]}, Actual: {all_actuals[-(i+1)]}")

            # Save the model if the test loss is the best we've seen so far
            if avg_test_loss < best_test_loss:
                best_test_loss = avg_test_loss
                print(f"Test loss improved, saving model...")
                model_save_path = os.path.join(save_path, f"best_model_epoch_{epoch+1}.pth")
                torch.save(model.state_dict(), model_save_path)
                print(f"Model saved to {model_save_path}")

    print("Training completed.")
    return train_losses, test_losses  # Return the losses for each epoch




In [95]:
train(train_loader,test_loader,model)

Epoch [1/20], Training Loss: 0.4222


Epoch [1/20], Test Loss: 1.6111

Last 5 Predictions and Actuals:
Predicted: [0.6601286], Actual: 0.8028689622879028
Predicted: [0.78201663], Actual: 1.8119971752166748
Predicted: [-0.11350755], Actual: 0.08764508366584778
Predicted: [-0.9148649], Actual: -1.169517993927002
Predicted: [-0.35942832], Actual: -0.7176188826560974
Test loss improved, saving model...
Model saved to /home/suneel/Desktop/chess-engine/models/basemodel1/best_model_epoch_1.pth


Epoch [2/20], Training Loss: 0.4219


Epoch [2/20], Test Loss: 1.6328

Last 5 Predictions and Actuals:
Predicted: [0.70395535], Actual: 0.8028689622879028
Predicted: [0.82400197], Actual: 1.8119971752166748
Predicted: [-0.14277957], Actual: 0.08764508366584778
Predicted: [-0.93730474], Actual: -1.169517993927002
Predicted: [-0.31030178], Actual: -0.7176188826560974


Epoch [3/20], Training Loss: 0.4216


...

Epoch [19/20], Training Loss: 0.4208


Epoch [19/20], Test Loss: 1.5746

Last 5 Predictions and Actuals:
Predicted: [0.68297625], Actual: 0.8028689622879028
Predicted: [0.92494094], Actual: 1.8119971752166748
Predicted: [-0.00751748], Actual: 0.08764508366584778
Predicted: [-0.7934909], Actual: -1.169517993927002
Predicted: [-0.272476], Actual: -0.7176188826560974


Epoch [20/20], Training Loss: 0.4205


Epoch [20/20], Test Loss: 1.5940

Last 5 Predictions and Actuals:
Predicted: [0.72864074], Actual: 0.8028689622879028
Predicted: [0.8897724], Actual: 1.8119971752166748
Predicted: [0.06381817], Actual: 0.08764508366584778
Predicted: [-0.8207586], Actual: -1.169517993927002
Predicted: [-0.2819755], Actual: -0.7176188826560974
Training completed.


([0.42218298639392854,
  0.4219494800107479,
  0.4215850016284784,
  0.4213745679262479,
  0.4214122645513217,
  0.4206773987111251,
  0.42041844150892893,
  0.4201096068973541,
  0.4202456603913307,
  0.42042678244928516,
  0.419678909400781,
  0.4202597007878621,
  0.42005210402234394,
  0.4200493460357984,
  0.420483204980135,
  0.4206784876169364,
  0.42076597462558746,
  0.42087323514445624,
  0.42076975457930565,
  0.42054796297629676],
 [1.611098958919525,
  1.6327864430885315,
  1.5810879185271263,
  1.646381567841053,
  1.6246791356306076,
  1.6080850962257385,
  1.61572068257761,
  1.6037245503044129,
  1.6024780660734177,
  1.6153107302756309,
  1.609990381336689,
  1.640928614792347,
  1.6300130306348801,
  1.5871292794246674,
  1.5889021974720954,
  1.5886252916212082,
  1.5504067338705063,
  1.5479840978155137,
  1.5745727281098365,
  1.5939625702962876])